In [1]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister
import time
from scipy.optimize import minimize
import matplotlib.pyplot as plt

import Entangler
import TensorNetwork
import hamiltonians
import TNOptimize
import uuid
import json
import utils

/home/alexey/quantumenv/lib/python3.6/site-packages/marshmallow/schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


### Initialize the tensor network

In [2]:
n_qubits = 6
depth = 2
q, c = QuantumRegister(n_qubits), ClassicalRegister(n_qubits)
ent = Entangler.IsingEntangler()
TN = TensorNetwork.Checkerboard(q, c, ent, depth=depth)
TN_UCC = TensorNetwork.UCCSD(q, c)

tol = 1e-7
method = "L-BFGS-B"

### All code loop together

In [4]:
#init params

J = 1
h_list = np.linspace(0, 2, num=3)
#h_list = [1.4]

start_vals = np.random.rand(TN.n_params) * 0.2

for h in h_list:
    print('''
    ------ h = {0:0.2f} -------
    '''.format(h))

    ham_dict = hamiltonians.ising_model(n_qubits, J, h)
    H = hamiltonians.explicit_hamiltonian(ham_dict)
    f = TNOptimize.build_objective_function(TN, explicit_H=H)
    
    
    n_iters = 0
    start_time = time.time()
    res = minimize(f, start_vals, options={'maxiter': 300}, callback=None, 
                   tol=tol, method=method)
    time_spent = time.time() - start_time
    print('Elapsed time: {0:5.2f}'.format(time_spent))
    
    circ = TN.construct_circuit(res.x)
    state = utils.get_state(circ)
    print(res.fun)
    print(res.nit)
    print(res.message)
    
    # Warm start
    start_vals = res.x
    
    
    

    ############## PACK EVERYTHING INTO A JSON AND SAVE ##########
    datetime = time.strftime("%Y-%m-%d_%H-%M-%S")

    problem = {'title': 'Ising model',
               'n_qubits': n_qubits,
               'h': h,
               'J': J,
               'Hamiltonian': ham_dict}

    solution = {'n_qubits': n_qubits,
                'TN': {'name': str(type(TN)),
                       'depth': depth,
                       'circuit': circ.qasm(),
                       'n_params': TN.n_params,
                       'params': res.x.tolist()},
                'E': res.fun,
                'state': utils.jsonize(state),
                'optimizer': {'method': method,
                              'n_iters': res.nit,
                              'message': str(res.message),
                              'time': time_spent,
                              'tol': tol}
                }

    data_entry = {'id': str(uuid.uuid4()),
                  'datetime': datetime,
                  'problem': problem,
                  'solution': solution,
                  'comment': '''Warm start from previous function value. '''}

    with open(('saved_states/TN_state_'
               + datetime
               + '.json'), 'w', encoding='utf-8') as out_file:
        json.dump(data_entry, out_file, indent=2)
    

    
    
    
    


    ------ h = 0.00 -------
    
Elapsed time:  1.56
0.7237068551775184
3
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'

    ------ h = 1.00 -------
    
Elapsed time:  0.19
1.260844682565788
1
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'

    ------ h = 2.00 -------
    
Elapsed time:  0.10
1.7979824808676705
0
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'


### Initialize the model and build a function that we will minimize

In [54]:
J = 1
h = 1.4
ham_dict = hamiltonians.ising_model(n_qubits, J, h)
H = hamiltonians.explicit_hamiltonian(ham_dict)

f = TNOptimize.build_objective_function(TN, explicit_H=H)
f_UCC = TNOptimize.build_objective_function(TN_UCC, explicit_H=H)

In [55]:
initial_guess_UCC = np.random.rand(TN_UCC.n_params)
start_time = time.time()
print(f_UCC(initial_guess_UCC))
time_spent = time.time() - start_time
print('Elapsed time (UCC): {0:5.2f}'.format(time_spent))

initial_guess = np.random.rand(TN.n_params)
start_time = time.time()
print(f(initial_guess))
time_spent = time.time() - start_time
print('Elapsed time (Checkers): {0:5.2f}'.format(time_spent))

0.03506074530642117
Elapsed time (UCC):  1.66
10.43023615453291
Elapsed time (Checkers):  0.11


In [3]:
n_iters = 0
def my_callback(xk):
    global n_iters
    n_iters += 1
    print("iteration number: ", n_iters) # , ", f = {0:0.4f}".format(f_UCC(xk)))

### Minimize the objective function using `scipy.optimize.minimize`

In [46]:
initial_guess = np.random.rand(TN.n_params)

start_time = time.time()
res = minimize(f, initial_guess, options={'maxiter': 300}, callback=None, tol=1e-6, method='L-BFGS-B')
time_spent = time.time() - start_time
print('Elapsed time: {0:5.2f}'.format(time_spent))

Elapsed time: 1223.26


In [47]:
res

      fun: -9.539425499024464
 hess_inv: <75x75 LbfgsInvHessProduct with dtype=float64>
      jac: array([-5.50492985e-04, -1.75663928e-03, -1.05409015e-03, -2.03641548e-03,
        2.48689958e-06,  6.13091800e-03,  4.43023396e-04, -1.78346227e-04,
        5.66515723e-03, -1.48965285e-03,  8.57625082e-04,  2.22506458e-03,
        2.44853027e-03,  7.56017471e-04, -1.32693856e-04, -7.27951033e-04,
        4.66187089e-03,  1.73994152e-03, -4.40536496e-05,  2.49080756e-03,
        2.81836776e-03, -8.48565662e-04,  5.82023318e-03, -2.94626545e-03,
       -2.13518092e-03,  1.30846445e-03,  1.50226498e-03, -1.52766688e-04,
        3.39461792e-04, -1.40190082e-03,  6.53201937e-03, -1.76889614e-03,
        1.20792265e-05,  2.91375812e-03, -1.12994059e-03, -4.72990536e-03,
        4.06430445e-04,  1.09992015e-03,  2.87592172e-04,  1.06652465e-03,
       -5.87974114e-04, -5.88329385e-04,  6.22080165e-04, -7.02549130e-04,
        7.20135063e-04,  2.12274642e-03, -3.49587026e-04,  3.97548661e-03,
 

In [48]:
res.x # parameters of the network

array([ 8.93225749e-01,  3.14352202e-01, -2.98270612e-01,  1.40328509e+00,
       -6.31458398e-01,  2.14563038e+00,  8.70951253e-01,  4.59257518e-01,
       -1.38902530e-01, -9.68865526e-01,  6.06834986e-01,  1.24316243e+00,
        8.43927491e-01,  7.67346500e-01,  3.78544302e-01, -2.54163374e-01,
        1.06434472e+00, -4.11062836e-02, -5.58000875e-01,  1.22439903e+00,
        1.24809034e+00,  1.35930401e+00,  1.57869304e+00, -2.80034305e-01,
       -1.38778800e-01,  7.48900355e-01,  1.38377123e+00,  1.34393619e+00,
        1.30941656e+00,  9.22419013e-01,  1.40004018e+00,  1.42272912e+00,
        1.54416156e+00,  1.69910040e+00,  5.32161567e-03,  6.32144250e-01,
        2.29296505e+00,  1.22539416e+00,  2.02050432e-02, -1.04677305e-02,
        1.24724232e+00,  1.72592097e+00,  3.62959185e-01,  8.59736261e-01,
       -6.77847037e-01, -1.66650380e-01,  1.06596544e-01, -1.59639365e+00,
       -2.55013620e-02,  2.19155441e-01, -1.49099592e-02,  1.39576999e-01,
        1.27068408e+00,  

### Attack the problem using the unitary coupled cluster

In [15]:
u = [0.3459796 ,  0.1881438 ,  0.43668297,  0.33544372,  0.58386988,
     1.28384011,  0.12040493,  0.46235707,  0.90650791,  0.47911129,
     0.75070291,  0.41647268,  0.12686378,  0.73158246,  0.95909617,
        0.89446071, -0.14138742,  0.65298177,  0.50326844,  0.56655752,
        0.07699172,  0.73290532,  1.04215793,  0.41971287,  0.46128677,
        0.11670666,  0.44174015,  0.10067137, -0.13700442,  0.2397051 ,
        0.35481272,  0.49468465,  0.39846009,  0.17959881,  0.42988137,
        0.56426357,  0.5604773 , -0.09674341,  0.13266187,  0.62295265,
        1.07880004,  0.41143078,  0.33357574,  0.35628562, -0.31656568,
        0.34138884, -0.08504385,  1.02617222,  0.14877449,  0.4058248 ,
        0.60268936,  0.47883013,  0.46631509,  0.37011117,  0.37441209,
        0.64153443,  0.48385105,  0.67036198,  1.02656032,  0.38497947,
        0.49395253,  1.08803193, -0.43112839,  0.61908109, -0.62300815,
        0.12788859,  0.74795541,  0.88781853, -1.4781325 ,  0.71087901,
        0.72470435,  0.70215431,  0.76330661,  0.10715642,  0.5563262 ,
        0.31461813,  1.09263033, -0.68533491,  1.17143661,  0.47699914,
       -0.02072033,  0.75188866, -0.3578969 , -0.08574258,  0.51519326,
       -0.06773303,  1.53664408,  0.03580986,  0.50118135,  1.75627204,
        1.76238346,  1.11824534,  0.74446303,  0.48293034,  0.63035599,
       -0.78819018,  0.47353042,  1.30494639,  0.56483804,  1.18536922,
       -0.77994623,  0.35772214,  1.03337631, -0.0266532 ,  0.42755639,
        0.58588125,  0.68642275,  0.37761533,  0.0586069 ,  0.56688103,
        0.88463212,  0.75903077,  0.89218869,  0.56639536,  0.93964048,
        0.79808512,  0.21291738, -0.07686197, -0.08359217,  1.2766838 ,
        0.73855921, -0.10239705,  0.75149293,  0.46440844,  1.09594198,
        0.67364488,  0.27854642,  0.77677523,  1.0681138 ,  0.46121453,
        0.79265738,  0.12110635,  0.75173654,  0.30675003, -0.03228069,
        0.99621482,  1.17519869,  0.11128786,  0.42632439,  1.24177216,
        0.77727181,  1.23824438, -0.02403463,  0.37073725, -0.14286294,
        0.63273043,  1.05556028,  1.03549292, -0.05293927,  0.48877569,
        0.19862382, -0.40321137,  1.05522313,  0.63367991,  0.97141938,
        1.09539006,  0.1138089 ,  0.44005676,  0.98565867,  0.13730818,
        1.18987905,  1.47569574, -0.29696842,  0.88049558,  0.2417701 ,
       -0.0776904 ,  0.05184482,  0.66308837,  0.21098826, -0.03880141,
        0.27592626,  0.82207614,  0.40923972,  0.02537691,  0.74568892,
        0.80708786,  0.74168328,  0.30425446,  0.56184231,  0.48899616,
        0.96792436,  0.51462362,  0.24965156,  0.4625598 ,  0.57868485,
        0.5015464 ,  0.64298487,  0.84139796,  0.86907525,  0.47483399,
        0.19761158,  0.39690963,  0.68048882,  0.31713468,  0.43899854,
        0.75082778,  0.95499483,  0.74729031]

In [21]:
initial_guess = np.random.rand(TN_UCC.n_params)

start_time = time.time()
res_UCC = minimize(f_UCC, u, options={'maxiter': 200}, callback=my_callback, tol=1e-6, method='L-BFGS-B')
time_spent = time.time() - start_time
print('Elapsed time: {0:5.2f}'.format(time_spent))
print(res_UCC.fun)

iteration number:  76
iteration number:  77
iteration number:  78
iteration number:  79
iteration number:  80
iteration number:  81
iteration number:  82
iteration number:  83
iteration number:  84
iteration number:  85
iteration number:  86
iteration number:  87
iteration number:  88
iteration number:  89
iteration number:  90
iteration number:  91
iteration number:  92
iteration number:  93
iteration number:  94
iteration number:  95
iteration number:  96
iteration number:  97
iteration number:  98
iteration number:  99
iteration number:  100
iteration number:  101
iteration number:  102
iteration number:  103
iteration number:  104
iteration number:  105
iteration number:  106
iteration number:  107
iteration number:  108
iteration number:  109
iteration number:  110
iteration number:  111
iteration number:  112
iteration number:  113
iteration number:  114
iteration number:  115
iteration number:  116
iteration number:  117
iteration number:  118
iteration number:  119
iteration nu

In [9]:
w, v = np.linalg.eigh(H)
print(w[0])

-9.545045186933953


### Construct the state that delivers this minimum

In [34]:
from qiskit import Aer, execute
sv_backend = Aer.get_backend("statevector_simulator")

def get_state(circ):
    job = execute(circ, sv_backend)
    result = job.result()
    state = result.get_statevector(circ)
    return state

In [35]:
circ = TN.construct_circuit(res.x)
state = get_state(circ)
print(state)

[-0.03781586-0.00731101j  0.07644851+0.01533371j  0.06820375+0.01350147j
 -0.07239399-0.01372635j  0.08720236+0.01454495j -0.14510867-0.02760466j
 -0.08303497-0.01577489j  0.07094538+0.01385552j  0.06955641+0.01123374j
 -0.14040041-0.02276286j -0.12733268-0.02330309j  0.13429993+0.02212136j
 -0.08367982-0.01195539j  0.13819057+0.02104669j  0.07930573+0.01281155j
 -0.06705836-0.01014078j  0.07789746+0.01399055j -0.17847831-0.03298345j
 -0.13799466-0.02546644j  0.16896268+0.02876292j -0.14555903-0.02491967j
  0.27709614+0.05306646j  0.13548354+0.0254304j  -0.13445131-0.02472636j
 -0.07506179-0.01102011j  0.17367402+0.02655737j  0.1343834 +0.02192591j
 -0.16580256-0.02453626j  0.0724565 +0.01209916j -0.1383911 -0.02511042j
 -0.06683151-0.01178928j  0.06647863+0.01094606j  0.06649922+0.01295131j
 -0.06708729-0.0110131j  -0.13472557-0.0233449j   0.07077744+0.01224732j
 -0.16413547-0.02574477j  0.1345316 +0.02143153j  0.1723019 +0.0279103j
 -0.0752781 -0.01414883j -0.1325656 -0.02612625j  0.

In [12]:
circ_UCC = TN_UCC.construct_circuit(res_UCC.x)
state_UCC = get_state(circ_UCC)
print(state_UCC)

[-0.03716488+3.79956192e-03j  0.07787749+1.54902388e-03j
  0.06731807-9.14466828e-03j -0.06521603+4.04714809e-03j
  0.07928421-6.74227977e-05j -0.18142754-7.49454853e-03j
 -0.06408793+9.52640127e-04j  0.07594844+6.90439840e-04j
  0.06355837-3.66843160e-03j -0.12408447+1.47519031e-02j
 -0.12876905+3.57337113e-02j  0.12509572-2.22421069e-02j
 -0.07240907+2.44949110e-03j  0.13991723-1.98878111e-03j
  0.06703237-1.48683010e-02j -0.06767952+8.09022664e-03j
  0.07534093-2.40640437e-03j -0.17090452-7.99501469e-03j
 -0.1313426 +2.67885291e-03j  0.14017371-5.03520272e-03j
 -0.18129826-7.37226823e-03j  0.42669104+1.94630113e-02j
  0.15246503-5.44697579e-03j -0.17784891-6.68464735e-04j
 -0.06085326+3.24982492e-03j  0.1310595 -1.29740384e-02j
  0.12319289-1.69800886e-02j -0.12469588+1.34770790e-02j
  0.0778373 -1.36956606e-03j -0.16693857-2.45487702e-04j
 -0.07068602+8.58694516e-03j  0.08471263-3.31267319e-03j
  0.06613889-7.80547511e-03j -0.06720513+2.25459967e-03j
 -0.13666986+2.61980591e-02j  0

In [2]:
res_UCC_data = """fun: -9.544878436036479
 hess_inv: <198x198 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.11928261e-04, -4.74642547e-04, -2.32525110e-04,  8.64019967e-04,
        1.43369761e-03, -8.22453217e-05,  5.70210545e-04, -8.78586093e-04,
       -3.11217718e-04,  1.06048503e-04,  1.02353681e-03,  3.53495011e-04,
        6.43929354e-04, -4.47997195e-04, -5.76605430e-04,  2.83684187e-04,
        3.85469434e-05,  5.04485342e-05, -7.89057708e-04, -7.08588743e-04,
        3.65218966e-04,  1.05515596e-03, -1.76925141e-04, -7.88524801e-04,
       -1.08357767e-04,  1.17061916e-04,  2.18136620e-04, -1.27808875e-03,
       -7.08588743e-04, -1.09245946e-04,  2.94697600e-04,  8.68638494e-04,
        8.73967565e-05,  3.65041330e-04,  1.17417187e-04,  2.94342328e-04,
        5.94724270e-04,  4.84412510e-04,  1.05515596e-03,  2.18491891e-04,
        8.68105587e-04,  5.95612448e-04, -1.52766688e-04, -1.76925141e-04,
       -1.27737820e-03,  8.73967565e-05,  4.84945417e-04, -1.52589053e-04,
        8.08952905e-04, -6.60094202e-04, -2.30038211e-03, -5.46940271e-04,
       -1.16440191e-03, -1.00452979e-03,  4.00213196e-04, -1.05853104e-03,
       -1.88116189e-04,  5.07682785e-04, -8.67750316e-04,  6.27053964e-04,
        5.25623989e-04, -2.10853557e-04, -6.21547258e-04, -5.63282754e-04,
        2.39275266e-04, -3.52251561e-04, -2.82618373e-04, -7.99360578e-05,
        2.32347475e-04, -1.15800702e-03,  1.08606457e-03,  6.87805368e-04,
        3.82982535e-04,  1.93622895e-05,  6.14441831e-04, -9.11803966e-04,
        1.74615877e-04, -2.46380694e-04, -1.33866251e-03, -6.67732536e-04,
       -3.42126327e-04, -1.14486198e-03, -1.61275437e-03, -7.17648163e-05,
        5.34505773e-04, -7.96518407e-04, -3.56337182e-04,  4.58300065e-04,
       -2.49400500e-04, -6.25455243e-04, -2.48547849e-03,  2.79420931e-04,
        9.17133036e-04,  2.24709140e-04,  2.11102247e-03, -2.89723801e-04,
        9.74509362e-04, -7.72715225e-04, -2.94519964e-04, -3.43014506e-04,
        6.42330633e-04,  5.79447601e-04, -8.65973959e-04, -4.29167812e-04,
       -1.73123738e-03,  8.26894109e-04, -4.93116659e-04,  1.17594823e-04,
       -2.43360887e-05,  9.04698538e-04, -2.61834998e-04,  1.15463195e-05,
        8.85513884e-04, -2.46913601e-05,  5.13544762e-04,  1.18571819e-03,
       -5.05728792e-04, -1.55786495e-04,  9.05231445e-04,  5.13367127e-04,
       -7.25819405e-04,  2.90434343e-04, -1.04982689e-04, -2.61834998e-04,
        1.18571819e-03, -7.25641769e-04, -1.82609483e-04, -6.61692923e-04,
        1.11910481e-05, -5.06084064e-04,  2.89901436e-04, -1.82964754e-04,
        3.71791486e-04,  8.85691520e-04, -1.55786495e-04, -1.05337961e-04,
       -6.61870558e-04,  3.71791486e-04, -1.43280943e-03, -2.21866969e-04,
       -3.95417032e-04,  8.79296636e-05, -4.80504525e-04,  3.06421555e-04,
        6.82298662e-04,  1.78523862e-04,  8.41460235e-04,  9.46798195e-04,
        1.22284405e-03,  2.21866969e-04,  2.27551311e-04, -1.06901155e-03,
       -1.45998769e-03, -1.68753900e-04, -7.93143329e-04, -6.19770901e-04,
        9.19442300e-04,  2.60058641e-04, -1.76392234e-04, -1.03383968e-03,
        1.34470213e-04, -2.19912977e-04, -5.47117907e-04,  1.38538070e-03,
       -5.52446977e-05, -1.18483001e-04, -7.30437932e-04,  8.83382256e-04,
       -5.16919840e-05, -6.05737682e-04,  6.84430290e-04,  1.56852309e-04,
        5.77315973e-05, -5.16919840e-05, -1.00701669e-03, -4.64162042e-04,
        5.36459765e-05,  1.45483625e-04, -6.05560047e-04, -1.00683906e-03,
        3.62732067e-04,  2.61302091e-04,  2.56328292e-04,  6.84430290e-04,
       -4.63984406e-04,  3.62732067e-04, -1.98596695e-04,  5.43565193e-05,
        1.56852309e-04,  5.36459765e-05,  2.61302091e-04, -1.98596695e-04,
        2.83328916e-04,  5.79092330e-05,  1.45483625e-04,  2.56150656e-04,
        5.43565193e-05,  2.83328916e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 14726
      nit: 71
   status: 0
  success: True
        x: array([ 0.32462166,  0.25127747,  0.46318922,  0.31958361,  0.58673205,
        1.29822455,  0.13063281,  0.36477223,  0.89128756,  0.52378377,
        0.764261  ,  0.39559835,  0.09926025,  1.03446439,  1.00222291,
        0.93305552,  0.01011956,  0.83792206,  0.54092217,  0.44262435,
        0.09138988,  0.82840496,  1.06211512,  0.4573768 ,  0.43720757,
        0.14791063,  0.4743797 ,  0.05666393, -0.26095814,  0.21560475,
        0.23130754,  0.49689457,  0.43287832,  0.19396765,  0.46106994,
        0.44076957,  0.55169209, -0.08087322,  0.22814359,  0.65559858,
        1.08099349,  0.40265222,  0.37316207,  0.3762421 , -0.36056563,
        0.37580445, -0.0691726 ,  1.06576431,  0.1645105 ,  0.3365082 ,
        0.56745787,  0.66564823,  0.72505696,  0.48259136,  0.33358149,
        0.73407482,  0.47025753,  0.67297585,  1.32117443,  0.35278988,
        0.74426396,  1.07827079, -0.68521745,  0.68765817, -0.83986703,
       -0.01261421,  0.66526979,  0.97115925, -1.57681663,  0.55294961,
        0.71613408,  0.71280723,  0.75338557, -0.00200269,  0.4419698 ,
        0.33277823,  0.93847094, -0.70614156,  1.24051337,  0.34142015,
       -0.14624319,  0.63386863, -0.42599612, -0.21750406,  0.59148169,
       -0.00244842,  1.63715284,  0.20926017,  0.42216774,  1.88031511,
        1.79919129,  1.16826316,  0.75386115,  0.43497924,  0.8082182 ,
       -0.84085088,  0.62156614,  1.18574731,  0.60235081,  1.40963646,
       -0.83235076,  0.28534376,  1.03385433,  0.06030411,  0.33082179,
        0.64803151,  0.75074948,  0.17291431,  0.09380416,  0.49101734,
        0.82225371,  0.69287402,  0.90152798,  0.6015995 ,  1.03020408,
        0.87595849,  0.26964296, -0.16678451, -0.15944704,  1.36725326,
        0.617903  , -0.20279363,  0.86096053,  0.4020236 ,  1.17381166,
        0.55299053,  0.25222573,  0.81462107,  1.00196071,  0.51794511,
        0.69226478,  0.09478675,  0.85121458,  0.31609756, -0.12220367,
        1.10567576,  1.21304744,  0.21076685,  0.40259053,  1.30353654,
        0.7456082 ,  1.53408064, -0.15903044,  0.30693771, -0.32788891,
        0.49400125,  1.22354555,  1.07221443, -0.03123931,  0.49308244,
        0.11529952, -0.48937255,  1.02954141,  0.72845032,  1.15618258,
        1.20298074,  0.10854127,  0.74421976,  0.76157504,  0.32214089,
        1.22365702,  1.58101311, -0.36168485,  0.82248481,  0.19909302,
       -0.03653638,  0.26912634,  0.64776891,  0.25556156, -0.105532  ,
        0.18778428,  0.81669242,  0.28382102,  0.06995148,  0.72658958,
        0.67289082,  0.70171882,  0.20930646,  0.49511351,  0.46989216,
        1.06655952,  0.67012285,  0.26475771,  0.37441906,  0.44449044,
        0.60018233,  0.68475192,  0.74677735,  0.86368774,  0.43486974,
        0.35311331,  0.43868039,  0.77147887,  0.19172037,  0.3440491 ,
        0.76593999,  0.86037478,  0.83827933])"""

In [ ]:
res.nit

### PACK EVERYTHING INTO A JSON AND SAVE

В этом блоке кое-что нужно править руками

In [33]:
circ = TN.construct_circuit(res.x)

In [44]:
datetime = time.strftime("%Y-%m-%d_%H-%M-%S")

problem = {'title': 'Ising model',
           'n_qubits': n_qubits,
           'h': h,
           'J': J,
           'Hamiltonian': ham_dict}

solution = {'n_qubits': n_qubits,
             'TN': {'name': str(type(TN)),
                    'depth': depth,
                    'circuit': circ.qasm(),
                    'n_params': TN.n_params,
                    'params': res.x.tolist()},
            'E': res.fun,
            'state': jsonize(state),
             'optimizer': {'method': "L-BFGS-B",
                           'n_iters': res.nit,
                           'message': str(res.message),
                           'time': 642.70,
                           'tol': 1e-6}
            }

data_entry = {'id': str(uuid.uuid4()),
              'datetime': datetime,
              'problem': problem,
              'solution': solution,
              'comment': 'From notebook'}

with open(('saved_states/TN_state_'
           + datetime
           + '.json'), 'w', encoding='utf-8') as out_file:
    json.dump(data_entry, out_file, indent=2)


In [50]:
%matplotlib tk

nq = [3, 4, 6]
ts = [137, 1042, 15141]
nql = np.log(nq)
tsl = np.log(ts)
plt.scatter(nql, tsl)

In [51]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(nql, tsl)

In [52]:
slope

6.775823717299451